In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from tqdm import tqdm

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import pickle
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.utils import shuffle
import gc

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)

In [3]:
X_Train = pd.read_pickle(ENV.application_train_cleaned.value)
print('Train shape: {}'.format(X_Train.shape))

X_Test = pd.read_pickle(ENV.application_test_cleaned.value)
print('Test shape: {}'.format(X_Test.shape))

X_ins = pd.read_pickle(ENV.installments_payments_clean.value)
print('Installment shape: {}'.format(X_ins.shape))

X_ins_fe = pd.read_pickle(ENV.installments_payment_clean_rnn.value)
print('Installment shape: {}'.format(X_ins_fe.shape))

Train shape: (307511, 122)
Test shape: (48744, 121)
Installment shape: (13605401, 8)
Installment shape: (997752, 13)


In [4]:
def get_embeddings_index(sorted_df,nor_ebd):
    embeddings_index={}
    words_values = sorted_df['words'].values
    for index in range(len(words_values)):
        embeddings_index  [words_values[index]] = nor_ebd[index,:]
    return embeddings_index

def create_document(sorted_df):
    #Create document
    ids = sorted_df.SK_ID_CURR.values
    words = sorted_df.words.values
    document_dicts = {}

    id_list = []
    document_list = []

    for index in range(len(ids)) :
        if document_dicts.get(ids[index]) is None:
            document_dicts[ids[index]] = []
        document_dicts[ids[index]].append(words[index])

    for key in document_dicts :
        document_dicts[key] = ' '.join(document_dicts[key])
        id_list.append(key)
        document_list.append(document_dicts[key])


    df_doc = pd.DataFrame({'SK_ID_CURR':id_list, 'text':document_list})  
    df_doc_mapping  = df_doc.set_index('SK_ID_CURR').text

    train = X_Train[['SK_ID_CURR','TARGET']].copy()
    test = X_Test[['SK_ID_CURR']].copy()
    train['text'] = train.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    test['text'] = test.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    return train,test


def get_train_ebdMat(train,test,embeddings_index):
    X_train = train["text"].str.lower()
    X_test = test["text"].str.lower()
    y_train = train["TARGET"].values
    tok=text.Tokenizer(num_words=max_features,lower=True,filters='')
    tok.fit_on_texts(list(X_train)+list(X_test))
    X_train=tok.texts_to_sequences(X_train)
    X_test=tok.texts_to_sequences(X_test)
    x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
    x_test=sequence.pad_sequences(X_test,maxlen=maxlen)
    print('...get ebd mat')
    word_index = tok.word_index
    #prepare embedding matrix
    num_words = min(max_features, len(word_index) + 1)
    print('num of words: {}'.format(num_words))
    embedding_matrix = np.zeros((num_words, embed_size))
    print(embedding_matrix.shape)
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return x_train,x_test,y_train,embedding_matrix,num_words

class_ratio =  sum(X_Train.TARGET ==0)/sum(X_Train.TARGET ==1)
class_ratio =  1
def get_rnn_model(num_words,embed_size,embedding_matrix):
    sequence_input = Input(shape=(maxlen, ))
    
    x = Embedding(num_words, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
    x = BatchNormalization()(x)
    x = SpatialDropout1D(0.2)(x)
#     x = Bidirectional(GRU(16, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Bidirectional(CuDNNGRU(8, return_sequences=True))(x)
    x = Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    x = BatchNormalization()(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool]) 
#     x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(sequence_input, preds)
#     Adam,RMSprop,Adagrad,Adadelta,Adamax,Nadam
#     model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-4),metrics=['accuracy'])
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    return model

def train_each_epoch(x,y,batch_size,model):
    x,y = shuffle(x,y)
    model.fit(x, y, 
              batch_size=batch_size, 
              epochs=1,
              verbose=1,
              class_weight={0:1,1:class_ratio})
    return model

def load_model(model,filepath):
    model.load_weights(filepath)
    return model

def save_model(model, filepath):
    model.save_weights(filepath)

def train_each_fold(x,y,x_val,y_val,model,filepath,reportpath,predspath,
                    batch_size=512,total_epoch=40,patience=5,saving=True):
    ROC_AUC_SCORE = []
    for epoch in range(total_epoch):  
        model = train_each_epoch(x,y,batch_size,model)
        y_pred = model.predict(x_val,batch_size=3000,verbose=1)
        score = roc_auc_score(y_val,y_pred)
        if len(ROC_AUC_SCORE) == 0:
            if saving:
                save_model(model,filepath)
            best_score = 0 
            if saving:
                print('saving preds...')
                pickle.dump(y_pred,open(predspath,'wb'))
        else:
            best_score = max(ROC_AUC_SCORE)
            if score >= best_score:
                if saving:
                    print('saving model to... {}'.format(filepath))
                    save_model(model,filepath)
                    print('saving preds...')
                    pickle.dump(y_pred,open(predspath,'wb'))
        ROC_AUC_SCORE.append(score)
        if saving:
            print('saving report to... {}'.format(reportpath))
            pickle.dump(ROC_AUC_SCORE,open(reportpath,'wb'))
        print('======= current {} / {}'.format(epoch,total_epoch))
        print('previous best roc is {}'.format(best_score))
        print('current roc is {}'.format(score))
        try:
            best_round = ROC_AUC_SCORE.index(best_score)
        except ValueError:
            best_round = -1
        if len(ROC_AUC_SCORE) > patience + best_round:
            print('reach patience! end loop')
            break
            
def train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=512,total_epoch=400,patience=30):
    train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
    val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

    for fold in range(0,len(train_fold_index)):
        print('!!!!!!!! Begin fold: {}'.format(fold))
        train_index = train_fold_index[fold]
        val_index = val_fold_index[fold]
        X_tra = x_train[train_index]
        y_tra = y_train[train_index]
        X_val = x_train[val_index]
        y_val = y_train[val_index]
        print('preparing train/val done!')
        print('before evaluating: {}'.format(model_file))
        model_file_evl = model_file.format(fold)
        report_file_evl = report_file.format(fold)
        pred_file_evl = pred_file.format(fold)
        pred_test_file_evl = pred_test_file.format(fold)
        model = get_rnn_model(num_words,embed_size,embedding_matrix)
        train_each_fold(X_tra, y_tra, X_val, y_val,
                        model,
                        filepath=model_file_evl,reportpath=report_file_evl,predspath=pred_file_evl,
                        batch_size=batch_size,total_epoch=total_epoch,patience=patience)
        gc.collect()
        #### predict test
        model = load_model(model,model_file_evl)
        test_preds = model.predict(x_test,batch_size=1500,verbose=1)
        pickle.dump(test_preds,open(pred_test_file_evl,'wb'))
        print('\n')


# Installment

In [5]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
trans_col = ['NUM_INSTALMENT_VERSION',
             'NUM_INSTALMENT_NUMBER',
             'DAYS_INSTALMENT',
             'DAYS_ENTRY_PAYMENT',
             'AMT_INSTALMENT',
             'AMT_PAYMENT']
for col in trans_col:
    print(col)
    X_ins[col] = qt.fit_transform(X_ins[col].values.reshape(-1,1))

NUM_INSTALMENT_VERSION
NUM_INSTALMENT_NUMBER
DAYS_INSTALMENT
DAYS_ENTRY_PAYMENT
AMT_INSTALMENT
AMT_PAYMENT


In [6]:


label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#13605401
max_features = 13605402
#372
maxlen = 75
embed_size = 6

sorted_df = X_ins.sort_values(['SK_ID_CURR','DAYS_INSTALMENT']).copy()
col = 'DAYS_INSTALMENT'
sorted_df[col] = qt.fit_transform(sorted_df[col].values.reshape(-1,1))
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embed_size = len(feature)
print('ebd size is {}'.format(embed_size))
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.installment_rnn.value
report_file = ENV.installment_report.value
pred_file = ENV.installment_preds.value
pred_test_file = ENV.installment_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=30)

start normalize
get embedding
ebd size is 6
create document
get embedding Mat
...get ebd mat
num of words: 13605402
(13605402, 6)
!!!!!!!! Begin fold: 0
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/installment/fold_{}.hdf5
Epoch 1/1
61503/61503 [==============================] - 0s 5us/step
saving preds...
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 0 / 500
previous best roc is 0
current roc is 0.5886229616329318
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving model to... ../LIB/../../data/rnn/installment/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 1 / 500
previous best roc is 0.5886229616329318
current roc is 0.5983691969486544
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving model to... ../LIB/../../data/rnn/installment/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/insta

Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving model to... ../LIB/../../data/rnn/installment/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 22 / 500
previous best roc is 0.6365269173491114
current roc is 0.6366605661517007
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 23 / 500
previous best roc is 0.6366605661517007
current roc is 0.6360692370488548
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 24 / 500
previous best roc is 0.6366605661517007
current roc is 0.6332296837607819
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 25 / 500
previous best roc is 0.6366605661

Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 45 / 500
previous best roc is 0.6433497583944999
current roc is 0.6391963245548089
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 46 / 500
previous best roc is 0.6433497583944999
current roc is 0.6399953156653333
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 47 / 500
previous best roc is 0.6433497583944999
current roc is 0.6409786744408927
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 48 / 500
previous best roc is 0.6433497583944999
current roc is 0.6423122320675347
Epoch 1/1
61503/61503 [=================

61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 69 / 500
previous best roc is 0.6461683882015024
current roc is 0.6450985647141194
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 70 / 500
previous best roc is 0.6461683882015024
current roc is 0.6461494248956965
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 71 / 500
previous best roc is 0.6461683882015024
current roc is 0.6420782341277811
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 72 / 500
previous best roc is 0.6461683882015024
current roc is 0.6445778826233423
Epoch 1/1
61503/61503 [===========================

61503/61503 [==============================] - 0s 3us/step
saving model to... ../LIB/../../data/rnn/installment/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 93 / 500
previous best roc is 0.6464739370218823
current roc is 0.6466040205651618
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 94 / 500
previous best roc is 0.6466040205651618
current roc is 0.6458903786778842
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 95 / 500
previous best roc is 0.6466040205651618
current roc is 0.6432118305633827
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 96 / 500
previous best roc is 0.6466040205651618
cur

...

Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 148 / 500
previous best roc is 0.6464511388144332
current roc is 0.6438502546586549
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 149 / 500
previous best roc is 0.6464511388144332
current roc is 0.6440674848689967
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 150 / 500
previous best roc is 0.6464511388144332
current roc is 0.6451638466699932
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 151 / 500
previous best roc is 0.6464511388144332
current roc is 0.6443236142912041
Epoch 1/1
61502/61502 [=============

61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 172 / 500
previous best roc is 0.6471980821569292
current roc is 0.6465341845543372
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 173 / 500
previous best roc is 0.6471980821569292
current roc is 0.6462242909353368
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 174 / 500
previous best roc is 0.6471980821569292
current roc is 0.6461068953452118
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving model to... ../LIB/../../data/rnn/installment/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 175 / 500
previous best roc is 0.6471980821569292

61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 196 / 500
previous best roc is 0.6476319603461148
current roc is 0.6461106129475729
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 197 / 500
previous best roc is 0.6476319603461148
current roc is 0.6454168154069246
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 198 / 500
previous best roc is 0.6476319603461148
current roc is 0.6454836382244628
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 199 / 500
previous best roc is 0.6476319603461148
current roc is 0.6462028098477637
Epoch 1/1
61502/61502 [=======================

61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 220 / 500
previous best roc is 0.6485035265436374
current roc is 0.6450560163116241
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 221 / 500
previous best roc is 0.6485035265436374
current roc is 0.6473432548907663
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 222 / 500
previous best roc is 0.6485035265436374
current roc is 0.6466087228433126
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 223 / 500
previous best roc is 0.6485035265436374
current roc is 0.6453767499238905
Epoch 1/1
61502/61502 [=======================

Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 245 / 500
previous best roc is 0.6487107810670989
current roc is 0.6465947926834923
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 246 / 500
previous best roc is 0.6487107810670989
current roc is 0.6469630269835152
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 247 / 500
previous best roc is 0.6487107810670989
current roc is 0.6449716867693482
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 248 / 500
previous best roc is 0.6487107810670989
current roc is 0.6452352061919691
Epoch 1/1
61502/61502 [=============

In [8]:
aaaaa

NameError: name 'aaaaa' is not defined

In [7]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

count    307511.000000
mean         37.746487
std          40.589065
min           1.000000
25%          11.000000
50%          23.000000
75%          48.000000
max         372.000000
Name: text, dtype: float64

# INSTALLMENT V2 - per previous application version

In [5]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
trans_col = ['count_VERSION',
             'count_INSTALLMENT',
             'DAY_INS_SPAN',
             'DAY_ENTRY_SPAN',
             'CNT_LATE_PAYMENT',
             'CNT_LESS_PAYMENT',
             'TOTAL_AMT_INSTALMENT',
             'TOTAL_AMT_PAYMENT',
             'INSTAL_START_DAY',
             'OWE_PORTION']
for col in trans_col:
    print(col)
    X_ins_fe[col] = qt.fit_transform(X_ins_fe[col].values.reshape(-1,1))

count_VERSION
count_INSTALLMENT
DAY_INS_SPAN
DAY_ENTRY_SPAN
CNT_LATE_PAYMENT
CNT_LESS_PAYMENT
TOTAL_AMT_INSTALMENT
TOTAL_AMT_PAYMENT
INSTAL_START_DAY
OWE_PORTION


In [6]:


label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#13605401
max_features = 13605402
#372
maxlen = 12

sorted_df = X_ins_fe.sort_values(['SK_ID_CURR','INSTAL_START_DAY']).copy()
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embed_size = len(feature)
print('ebd size is {}'.format(embed_size))
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.installment_rnn.value
report_file = ENV.installment_report.value
pred_file = ENV.installment_preds.value
pred_test_file = ENV.installment_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=35)

start normalize
get embedding
ebd size is 11
create document
get embedding Mat
...get ebd mat
num of words: 997754
(997754, 11)
!!!!!!!! Begin fold: 0
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/installment/fold_{}.hdf5
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving preds...
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 0 / 500
previous best roc is 0
current roc is 0.5798514086719732
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/installment/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 1 / 500
previous best roc is 0.5798514086719732
current roc is 0.5904538567927875
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/installment/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/install

61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 20 / 500
previous best roc is 0.6358633110426912
current roc is 0.6347454641580249
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 21 / 500
previous best roc is 0.6358633110426912
current roc is 0.63275281239419
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 22 / 500
previous best roc is 0.6358633110426912
current roc is 0.6347165941658871
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 23 / 500
previous best roc is 0.6358633110426912
current roc is 0.6356758013583853
Epoch 1/1
61503/61503 [=============================

61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 44 / 500
previous best roc is 0.63810742836483
current roc is 0.6362585831645156
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 45 / 500
previous best roc is 0.63810742836483
current roc is 0.6359828799404307
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 46 / 500
previous best roc is 0.63810742836483
current roc is 0.6377545932304549
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 47 / 500
previous best roc is 0.63810742836483
current roc is 0.638014383705115
Epoch 1/1
61503/61503 [==============================] - 0s

Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 68 / 500
previous best roc is 0.6396659219315814
current roc is 0.6391836972958549
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 69 / 500
previous best roc is 0.6396659219315814
current roc is 0.6393475145049383
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 70 / 500
previous best roc is 0.6396659219315814
current roc is 0.6396096866015116
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 71 / 500
previous best roc is 0.6396659219315814
current roc is 0.6376825688948701
Epoch 1/1
61503/61503 [=================

61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/installment/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 92 / 500
previous best roc is 0.6400149407320475
current roc is 0.6408043730738991
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 93 / 500
previous best roc is 0.6408043730738991
current roc is 0.6400527812609397
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 94 / 500
previous best roc is 0.6408043730738991
current roc is 0.6386964027466127
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_0.pkl
======= current 95 / 500
previous best roc is 0.6408043730738991
cur

...

61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 16 / 500
previous best roc is 0.6324794181527023
current roc is 0.6312813737793427
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 17 / 500
previous best roc is 0.6324794181527023
current roc is 0.6312843753454513
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 18 / 500
previous best roc is 0.6324794181527023
current roc is 0.6308595054709365
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 19 / 500
previous best roc is 0.6324794181527023
current roc is 0.6316427948859331
Epoch 1/1
61502/61502 [===========================

61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 39 / 500
previous best roc is 0.6368724669132667
current roc is 0.6360433656434927
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 40 / 500
previous best roc is 0.6368724669132667
current roc is 0.6350268599663669
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/installment/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 41 / 500
previous best roc is 0.6368724669132667
current roc is 0.6374787452956057
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 42 / 500
previous best roc is 0.6374787452956057
cur

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 63 / 500
previous best roc is 0.6392429374740654
current roc is 0.6390545566536419
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 64 / 500
previous best roc is 0.6392429374740654
current roc is 0.6390621093895671
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 65 / 500
previous best roc is 0.6392429374740654
current roc is 0.638835986587585
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 66 / 500
previous best roc is 0.6392429374740654
current roc is 0.6375035208732089
Epoch 1/1
61502/61502 [==================

61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 87 / 500
previous best roc is 0.6403106740985154
current roc is 0.6374851570747753
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 88 / 500
previous best roc is 0.6403106740985154
current roc is 0.6381463568842141
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 89 / 500
previous best roc is 0.6403106740985154
current roc is 0.6383065067096647
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 90 / 500
previous best roc is 0.6403106740985154
current roc is 0.6387288252529871
Epoch 1/1
61502/61502 [===========================

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 112 / 500
previous best roc is 0.6414465770125631
current roc is 0.6393697807647436
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 113 / 500
previous best roc is 0.6414465770125631
current roc is 0.6373929529418949
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 114 / 500
previous best roc is 0.6414465770125631
current roc is 0.639866834687828
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/installment/report_fold_4.pkl
======= current 115 / 500
previous best roc is 0.6414465770125631
current roc is 0.6393054857721572
Epoch 1/1
61502/61502 [==============

In [10]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

count    307511.000000
mean          2.826605
std           2.032831
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max          26.000000
Name: text, dtype: float64

In [7]:
model = get_rnn_model(num_words,embed_size,embedding_matrix)

In [9]:
print(num_words)
print(embed_size)
print(embedding_matrix.shape)

13605402
6
(13605402, 6)


In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 75)           0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 75, 6)        81632412    input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_11 (BatchNo (None, 75, 6)        24          embedding_6[0][0]                
__________________________________________________________________________________________________
spatial_dropout1d_6 (SpatialDro (None, 75, 6)        0           batch_normalization_11[0][0]     
__________________________________________________________________________________________________
bidirectio